# IS620 Group Project

<b>Group project: Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can. Begin by splitting the Names Corpus into three subsets: 500 words for the test set, 500 words for the dev- test set, and the remaining 6900 words for the training set. Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set. How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?</b>

<font color= blue> <b>Group Members:- Aaron Palumbo, Brian Chu,  David Stern, Partha Banerjee;  Rohan Fray, Tulasi Ramarao;</b></font>

## Dependencies

In [349]:
import nltk
import numpy as np
import pandas as pd
import string
import networkx as nx
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from __future__ import division
%matplotlib inline

## Split Data

In [350]:
names = nltk.corpus.names
maleNames = names.words('male.txt')
femaleNames = names.words('female.txt')

* Training Data

    * Train Set: Used to train the model

    * Validation Set: Model Selection

* Test Set: Measure Final Model performance (Only use this once at the end)

There are different numbers of male and female names:

In [351]:
print "Number of male names: {}".format(len(maleNames))
print "Number of female names: {}".format(len(femaleNames))

Number of male names: 2943
Number of female names: 5001


We will have to do our splits separately. We will split the data with the goal of maintaining the same ratio of male to female in our train, validation, and test sets.

In [352]:
perMale = 1.0 * len(maleNames) / (len(maleNames) + len(femaleNames))
perMale

# total names
numNames = len(names.words())
# number used for testing
numTesting = 1000
# slit between final test and validation
perTest = 0.5

# numbers for data splitting
numTestingMale = int(perMale * numTesting)
numTestingFemale = numTesting - numTestingMale

numTestMale = int(numTesting * perTest * perMale)
numTestFemale = int(numTesting *  perTest - numTestMale)

In [353]:
maleTrain, maleTesting = train_test_split(
    maleNames, test_size=numTestingMale, random_state=5)
maleVal, maleTest = train_test_split(
    maleTesting, test_size=numTestMale, random_state=6)

femaleTrain, femaleTesting = train_test_split(
    femaleNames, test_size=numTestingFemale, random_state=7)
femaleVal, femaleTest = train_test_split(
    femaleTesting, test_size=numTestFemale, random_state=8)

# Check numbers
print "Val Set   = {} (Should be 500)".format(len(maleVal) + len(femaleVal))
print "Test Set  = {} (Should be 500)".format(len(maleTest) + len(femaleTest))
print "Train Set = {} (Should be >6900)".format(len(maleTrain) + len(femaleTrain))


Val Set   = 500 (Should be 500)
Test Set  = 500 (Should be 500)
Train Set = 6944 (Should be >6900)


In [354]:
train = pd.DataFrame({'name': maleTrain + femaleTrain,
                      'sex': (['male'] * len(maleTrain) + 
                              ['female'] * len(femaleTrain))})
validation = pd.DataFrame({'name': maleVal + femaleVal,
                           'sex': (['male'] * len(maleVal) +
                                   ['female'] * len(femaleVal))})
test = pd.DataFrame({'name': maleTest + femaleTest,
                     'sex': (['male'] * len(maleTest) +
                             ['female'] * len(femaleTest))})

In [355]:
# Just to make sure we all see the same thing
print train.loc[56, :]
print
print validation.loc[38, :]
print
print test.loc[486, :]

name    Kristos
sex        male
Name: 56, dtype: object

name    Orton
sex      male
Name: 38, dtype: object

name      Shea
sex     female
Name: 486, dtype: object


Names should be Kristos, Orton, and Shea

----

Use the above code to start another notebook to explore an algorithm. Make sure to use the splits as defined above and to not use the final test set to tune your model. =-)

-----

## Random Forest

#####The n_estimators and max_features are the two biggest tweaks to make to Random Forests

In [371]:
clf = RandomForestClassifier(n_jobs=2, n_estimators=3, max_features="auto",random_state=620)

In [372]:
def createFactorDF(array):
    le = preprocessing.LabelEncoder()
    le.fit(array)
    X=le.transform(array) 
    return X, le

In [373]:
first, firstLE = createFactorDF([(word[-1:]) for word, gender in train.values])

In [374]:
second, secondLE = createFactorDF([(word[1:]) for word, gender in train.values])

In [375]:
#third, thirdLE = createFactorDF([(len(word)) for word, gender in train.values])

In [376]:
Y, YLE = createFactorDF([(gender) for word, gender in train.values])

In [377]:
p= pd.DataFrame(first)
p['second']=second
#p['third']=third

In [378]:
clf_fit = clf.fit(p,pd.Series(Y))

In [379]:
clf_fit

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=2,
            oob_score=False, random_state=620, verbose=0, warm_start=False)

In [380]:
first, firstLE = createFactorDF([(word[-1:]) for word, gender in validation.values])
second, secondLE = createFactorDF([(word[1:]) for word, gender in validation.values])
#third, thirdLE = createFactorDF([(len(word)) for word, gender in validation.values])
Y, YLE = createFactorDF([(gender) for word, gender in validation.values])
p2= pd.DataFrame(first)
p2['second']=second
#p2['third']=third

In [381]:
p2['pred_Y']=clf_fit.predict(p2)

In [382]:
p2['Y']=Y

In [383]:
pd.crosstab(p2['Y'], p2['pred_Y'], rownames=['actual'], colnames=['preds'])

preds,0,1
actual,,
0,157,158
1,47,138


In [386]:
accuracy = (157+138)/(157+158+47+138)
accuracy

0.59